<p>Class:  USC Viterbi Data Analytics Bootcamp</p>
<p>Team:  Analyticus (aka Team 5)</p>
<p>Module:  pull_cdc_state_5_years.py</p>
<p>Version:  March 31, 2018
<p>Input:  CDC Influenza-Like-Illness CSV File for last five flu seasons.</p>
<p>Output:  CDC json file with flu cases for the last five flu seasons.</p>

In [1]:
# Import dependances.
import json
import csv
import pandas as pd

In [2]:
# Load CDC data into a dataframe.
df = pd.read_csv('../data/cdc_state_5_years.csv', skiprows=[0])

In [3]:
# Inspect the dataframe.
df.head()

,REGION TYPE,REGION,YEAR,WEEK,% WEIGHTED ILI,%UNWEIGHTED ILI,AGE 0-4,AGE 25-49,AGE 25-64,AGE 5-24,AGE 50-64,AGE 65,ILITOTAL,NUM. OF PROVIDERS,TOTAL PATIENTS
0,States,Alabama,2012,40,X,1.49404,X,X,X,X,X,X,238,39,15930
1,States,Alaska,2012,40,X,1.19893,X,X,X,X,X,X,27,8,2252
2,States,Arizona,2012,40,X,0.756794,X,X,X,X,X,X,132,32,17442
3,States,Arkansas,2012,40,X,0.471698,X,X,X,X,X,X,14,17,2968
4,States,California,2012,40,X,2.10133,X,X,X,X,X,X,720,123,34264


In [4]:
# Select only columns needed.
df2 = df[['YEAR', 'WEEK', 'REGION', 'ILITOTAL']]

In [5]:
# Inspect dataframe
df2.head()

,YEAR,WEEK,REGION,ILITOTAL
0,2012,40,Alabama,238
1,2012,40,Alaska,27
2,2012,40,Arizona,132
3,2012,40,Arkansas,14
4,2012,40,California,720


In [6]:
# Normalize CDC year and weeks to HHS year and weeks.
df3 = pd.DataFrame()
for df2_index in df2.index:
    if df2.loc[df2_index, 'WEEK'] > 39:
        df3.loc[df2_index, 'YEAR'] = df2.loc[df2_index, 'YEAR']
        df3.loc[df2_index, 'WEEK'] = df2.loc[df2_index, 'WEEK'] - 39
    else:
        df3.loc[df2_index, 'WEEK'] = df2.loc[df2_index, 'WEEK'] + 13
        df3.loc[df2_index, 'YEAR'] = df2.loc[df2_index, 'YEAR'] - 1
    df3.loc[df2_index, 'REGION'] = df2.loc[df2_index, 'REGION']
    if df2.loc[df2_index, 'ILITOTAL'] == 'X':
        df3.loc[df2_index, 'ILITOTAL'] = 0
    else:
        df3.loc[df2_index, 'ILITOTAL'] = df2.loc[df2_index, 'ILITOTAL']
    

In [7]:
# Inspect the dataframe.
df3.head()

,YEAR,WEEK,REGION,ILITOTAL
0,2012.0,1.0,Alabama,238
1,2012.0,1.0,Alaska,27
2,2012.0,1.0,Arizona,132
3,2012.0,1.0,Arkansas,14
4,2012.0,1.0,California,720


In [8]:
# Sort the CDC dataframe into year, week, and region sequence to aid analysis.
df4 = df3.sort_values(['YEAR', 'WEEK', 'REGION'])

In [9]:
# Inspect CDC dataframe.
df4.head()

,YEAR,WEEK,REGION,ILITOTAL
0,2012.0,1.0,Alabama,238
1,2012.0,1.0,Alaska,27
2,2012.0,1.0,Arizona,132
3,2012.0,1.0,Arkansas,14
4,2012.0,1.0,California,720


In [10]:
# Create a save point to avoiding the current dataframe.
df5 = df4

In [11]:
# Change the type for year and week to type integer.
df5['YEAR'] = df5['YEAR'].astype(int)
df5['WEEK'] = df5['WEEK'].astype(int)
df5['ILITOTAL'] = df5['ILITOTAL'].astype(int)

In [12]:
# Check for type integer for year and week.
df5.head()

,YEAR,WEEK,REGION,ILITOTAL
0,2012,1,Alabama,238
1,2012,1,Alaska,27
2,2012,1,Arizona,132
3,2012,1,Arkansas,14
4,2012,1,California,720


In [13]:
# Calculate total flu cases per state per year.
# This divided into the flu cases per state per week will give the percent flu for the week.
df6 = pd.DataFrame(df5.groupby(['YEAR','REGION']).agg({'ILITOTAL': 'sum'}))

In [15]:
# Inspect the sum of flu cases per year per state.
df6.head()

ILITOTAL
YEAR REGION              
2012 Alabama        18285
     Alaska           998
     Arizona        13096
     Arkansas        2441
     California     41701

In [16]:
# Calculate the percent flu cases per week for the year for each state.
for df5_index in df5.index:
    
    year_index = df5.loc[df5_index, 'YEAR']
    region_index = df5.loc[df5_index, 'REGION']
    
    state_total = df6.loc[year_index].loc[region_index][0]
    
    if state_total > 0:
        df5.loc[df5_index, 'FLU_PERCENT'] = (df5.loc[df5_index, 'ILITOTAL'] / state_total) * 100
    else:
        df5.loc[df5_index, 'FLU_PERCENT'] = 0.0


In [18]:
# Validate the calculation by inspecting Alabama for 2013.
df7 = df5.loc[(df5['REGION'] == 'Alabama') & (df5['YEAR'] == 2013)]

In [19]:
# Inspect dataframe for Alabama 2013.
df7.head()

,YEAR,WEEK,REGION,ILITOTAL,FLU_PERCENT
2600,2013,1,Alabama,201,1.435202
2650,2013,2,Alabama,217,1.549447
2700,2013,3,Alabama,243,1.735095
2750,2013,4,Alabama,452,3.227419
2800,2013,5,Alabama,435,3.106034


In [21]:
# Add up the calculated flu percentages for Alabama 2013.
df8 = pd.DataFrame(df7.groupby(['YEAR']).agg({'FLU_PERCENT': 'sum'}))

In [22]:
# One hundred percent indicates all the calculated percentages add up correctly.
df8

,FLU_PERCENT
YEAR,
2013,100.0


In [23]:
# Inspect the CDC dataframe.
df5.head()

,YEAR,WEEK,REGION,ILITOTAL,FLU_PERCENT
0,2012,1,Alabama,238,1.301613
1,2012,1,Alaska,27,2.705411
2,2012,1,Arizona,132,1.007941
3,2012,1,Arkansas,14,0.573535
4,2012,1,California,720,1.726577


In [24]:
# Rename the CDC dataframe columns to match the HHS columns.
df9 = df5.rename(columns = {"YEAR":"year", "WEEK":"week", "REGION":"state", "ILITOTAL":"flu_cases", "FLU_PERCENT":"flu_percent"})

In [25]:
# Inspect the new column names.
df9.head()

,year,week,state,flu_cases,flu_percent
0,2012,1,Alabama,238,1.301613
1,2012,1,Alaska,27,2.705411
2,2012,1,Arizona,132,1.007941
3,2012,1,Arkansas,14,0.573535
4,2012,1,California,720,1.726577


In [26]:
# Write the CDC dataframe to a json file to be used by the plot modules.
df9.to_json('../data/cdc_state_5_years.json')

In [27]:
# Begin the validation of the file just written by loading it into a dataframe.
dfA = pd.read_json('../data/cdc_state_5_years.json')

In [28]:
# Sort the CDC data by year, state, and week to aid in inspection.
dfB = dfA.sort_values(by=['year', 'state', 'week'])

In [29]:
# Validate CDC data.
dfB.head()

,flu_cases,flu_percent,state,week,year
0,238,1.301613,Alabama,1,2012
50,288,1.575062,Alabama,2,2012
100,265,1.449275,Alabama,3,2012
150,261,1.427400,Alabama,4,2012
200,275,1.503965,Alabama,5,2012


In [31]:
# Validate CDC data.
dfB.tail()

,flu_cases,flu_percent,state,week,year
14099,246,6.232582,Wyoming,21,2017
14149,194,4.915125,Wyoming,22,2017
14199,194,4.915125,Wyoming,23,2017
14249,182,4.611097,Wyoming,24,2017
14299,138,3.496326,Wyoming,25,2017
